In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import TruncatedSVD

# pd.options.display.max_columns = 10 
# pd.options.display.width = 134
# pd.options.display.max_rows = 20

# import warnings
# warnings.filterwarnings("ignore", category=DeprecationWarning)



In [2]:
data_folder = './dataset/'


genome_scores_df=pd.read_csv(data_folder+'genome_scores.csv') # Large (500MB)
movies_df=pd.read_csv(data_folder+'movies.csv')
validation = pd.read_csv(data_folder+'validation.csv') 

df_train=pd.read_csv(data_folder+'train_with_valid.csv') # Large 500MB
test=pd.read_csv(data_folder+'test.csv') # Large 500MB


In [ ]:
# matrix = pd.concat([train,test]).drop_duplicates(subset = ['userId','movieId'],keep = 'first')
# matrix = matrix.pivot('userId','movieId','rating')

# movie_means = matrix.mean()
# user_means = matrix.mean(axis=1)
# #Mean shifting
# mzm = matrix-movie_means
# mz = mzm.fillna(0)
# #To get locations where we have ratings
# mask = -mzm.isnull()


/home/vishwesh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [3]:
#Due to adding of validation, we have some duplicates
movie_matrix = pd.concat([df_train,test]).drop_duplicates(subset = ['userId','movieId'],keep = 'first')
#Creates a movie matrix of #numofusers vs #noofmovies
movie_matrix = movie_matrix.pivot('userId','movieId','rating')

movie_means = movie_matrix.mean()
user_means = movie_matrix.mean(axis=1)
#Mean shifting
movie_shifted_temp = movie_matrix-movie_means
movie_shifted = movie_shifted_temp.fillna(0)
#To get locations where we have ratings
mask = -movie_shifted_temp.isnull()


NameError: name 'df_train' is not defined

In [ ]:
def repeated_matrix_reconstruction(num_pcs,num_iterations):

    SVD = TruncatedSVD(n_components=num_pcs,random_state=42)
    for i in range(num_iterations):
        SVD.fit(movie_shifted)
        #For the ease of applying masks we work with pandas
        movie_represented =  pd.DataFrame(SVD.inverse_transform(SVD.transform(movie_shifted)),columns=movie_shifted.columns,index=movie_shifted.index)
        loss = mean_squared_error(movie_represented[mask].fillna(0),movie_shifted_temp[mask].fillna(0))
        #To just update the non-zero values of movie_reprented values to the true ratings
        movie_represented[mask] = movie_shifted_temp[mask]
    
    #Mean shifting it  back
    movie_represented = movie_represented + movie_means
    movie_represented = movie_represented.clip(lower=0.5,upper=5)
    return movie_represented

In [ ]:
# iteration = 0
# mse_last = 999
# while iteration<10:
#     iteration += 1
#     svd = TruncatedSVD(n_components=20,random_state=42)
#     svd.fit(mz)
#     mzsvd = pd.DataFrame(svd.inverse_transform(svd.transform(mz)),columns=mz.columns,index=mz.index)

#     mse = mean_squared_error(mzsvd[mask].fillna(0),mzm[mask].fillna(0))
#     print('%i %.5f %.5f'%(iteration,mse,mse_last-mse))
#     mzsvd[mask] = mzm[mask]

#     mz = mzsvd
#     if mse_last-mse<0.00001: break
#     mse_last = mse

# m_20 = mz+movie_means
# m_20 = m_20.clip(lower=0.5,upper=5)


In [7]:
# iteration = 0
# mse_last = 999
# while iteration<10:
#     iteration += 1
#     svd = TruncatedSVD(n_components=15,random_state=42)
#     svd.fit(mz)
#     mzsvd = pd.DataFrame(svd.inverse_transform(svd.transform(mz)),columns=mz.columns,index=mz.index)

#     mse = mean_squared_error(mzsvd[mask].fillna(0),mzm[mask].fillna(0))
#     print('%i %.5f %.5f'%(iteration,mse,mse_last-mse))
#     mzsvd[mask] = mzm[mask]

#     mz = mzsvd
#     if mse_last-mse<0.00001: break
#     mse_last = mse

# m_15 = mz+movie_means
# m_15 = m_15.clip(lower=0.5,upper=5)


1 0.02925 998.97075
2 0.02905 0.00020
3 0.02890 0.00015
4 0.02878 0.00012
5 0.02867 0.00010
6 0.02858 0.00009
7 0.02850 0.00008
8 0.02843 0.00007
9 0.02836 0.00007
10 0.02830 0.00006


In [8]:
#With 15 PCs
representative_matrix_15 = repeated_matrix_reconstruction(15,10)
#With 20 PCs
representative_matrix_20 = repeated_matrix_reconstruction(20,10)


#bagging
representative_matrix = (representative_matrix_15+representative_matrix_20)/2

In [9]:
df_best_tillnow = pd.read_csv("./ensemble_models2/submission_emsembling_8114_withval.csv")

In [10]:

svd_prediction['rating'] = svd_prediction.apply(lambda x:representative_matrix[representative_matrix.index==representative_matrix.userId][representative_matrix.movieId].values[0],axis=1)

# There are some movies who did not have enough info to make prediction, so just used average value for user
# missing = np.where(test.rating.isnull())[0]
# test.ix[missing,'rating'] = user_means[test.loc[missing].userId].values

indices=np.where(svd_prediction.rating.isna()) 
svd_prediction.rating.iloc[indices] = df_best_tillnow.Prediction.iloc[indices]

rating_trunc = svd_prediction.rating

df_test=pd.read_csv(data_folder+'test.csv') # Large 500MB
predictions = np.zeros(len(df_test))
for i in range(len(df_test)):
    userid =  df_test.iloc[i,0]
    movieid = df_test.iloc[i,1]
    rating = PRED[i]
    if rating>5:
        rating = 5
    if rating<0.5:
        rating =0.5
    predictions[i] = rating
df_submission = pd.read_csv(data_folder+'dummy_submission.csv')
df_submission.Prediction = predictions
df_submission.to_csv('./ensemble_models2/Submission_truncSVD20_15_with_val.csv',index=False)

/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [13]:
df_trunc = pd.read_csv("./ensemble_models2/Submission_truncSVD20_15_with_val.csv")
df_latent = pd.read_csv("./ensemble_models2/Submission_pq_with_val.csv")
df_regression = pd.read_csv("./ensemble_models2/Submission_regression_final.csv")
df_nn =  pd.read_csv("./ensemble_models2/Submission_regression_NN.csv")


PRED = (8*df_trunc.Prediction + df_latent.Prediction + df_regression.Prediction + 4*df_nn.Prediction)/14
PRED = np.around(PRED,1)


In [14]:
df_test=pd.read_csv(data_folder+'test.csv') # Large 500MB

predictions = np.zeros(len(df_test))
for i in range(len(df_test)):
    userid =  df_test.iloc[i,0]
    movieid = df_test.iloc[i,1]
    rating = PRED[i]
    if rating>5:
        rating = 5
    if rating<0.5:
        rating =0.5
    predictions[i] = rating
df_submission = pd.read_csv(data_folder+'dummy_submission.csv')
df_submission.Prediction = predictions
df_submission.to_csv('./ensemble_models2/submission_emsembling_8114_withval_new.csv',index=False)